In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
# df_from_csv = pd.read_csv('dataset/auction_data_added_testing.csv')
df_from_csv = pd.read_csv('dataset/auction_data_added_training.csv')
df_from_csv.fillna(0, inplace=True) # fill for auctions with only 1 bid

## Explore Data

In [4]:
df_from_csv.head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,outcome,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid
0,0,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.157894736,0.000000,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,1,0
1,1,25,668d393e858e8126275433046bbd35c6tywop,38v0a,furniture,phone4,9759243315789473,bd,147.64.206.30,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.315789473,0.000000,3.931033,4.550885,0.0,0.947368,2.421053,5.210526,45.052632,2.421053,11.106156,3471,1,0
2,2,39,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone35,9759243368421052,in,75.11.239.20,i92xwb95qdo901j,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.210526,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,0,0
3,3,42,668d393e858e8126275433046bbd35c6tywop,jkyzt,furniture,phone38,9759243368421052,in,215.203.78.242,s4hrzaqd5vbq2he,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.000000,3.523134,4.137105,0.0,0.842105,2.105263,4.631579,60.473684,2.105263,17.341241,3872,1,0
4,4,68,668d393e858e8126275433046bbd35c6tywop,h2xrf,furniture,phone57,9759243526315789,in,134.198.237.242,1euklxshlldto4z,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.526315789,0.000000,2.897465,3.318102,0.0,0.684211,1.842105,3.894737,31.000000,1.842105,10.754362,4709,1,0


In [5]:
df_from_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3071224 entries, 0 to 3071223
Data columns (total 27 columns):
index                                  int64
bid_id                                 int64
bidder_id                              object
auction                                object
merchandise                            object
device                                 object
time                                   int64
country                                object
ip                                     object
url                                    object
payment_account                        object
address                                object
outcome                                float64
date_time                              object
time_delta_auction_bids                float64
time_delta_auction_bids_mean           float64
time_delta_auction_bids_std            float64
time_delta_auction_bids_min            float64
time_delta_auction_bids_25%            float64
time_delta

In [6]:
# bots = df_from_csv.loc[df_from_csv['outcome'] == 1]
bots = pd.read_csv('dataset/bots.csv')
bots.fillna(0, inplace=True) # fill for auctions with only 1 bid
bot_count = len(bots['bidder_id'].unique())
bot_ratio = bot_count / len(df_from_csv['bidder_id'].unique())
bot_bids = bots.shape[0]
bot_bids_ratio = bot_bids / df_from_csv.shape[0]
# bots.to_csv('dataset/bots.csv', index=None)

In [7]:
print('bots: {}, % {:.2f} of total bidders are bots for training'.format(bot_count, bot_ratio))
print('bot bids: {}, % {:.2f} of total bot bids for training'.format(bot_bids, bot_bids_ratio))

bots: 103, % 0.05 of total bidders are bots for training
bot bids: 412416, % 0.13 of total bot bids for training


In [8]:
for column in bots.columns:
    print(column + ': {}'.format(len(bots[column].unique())))

index: 412416
bid_id: 412416
bidder_id: 103
auction: 6390
merchandise: 7
device: 3078
time: 313212
country: 192
ip: 230125
url: 54865
payment_account: 103
address: 103
outcome: 1
date_time: 313212
time_delta_auction_bids: 16493
time_delta_auction_bids_mean: 6171
time_delta_auction_bids_std: 5991
time_delta_auction_bids_min: 1463
time_delta_auction_bids_25%: 4627
time_delta_auction_bids_50%: 4766
time_delta_auction_bids_75%: 5678
time_delta_auction_bids_max: 6079
time_delta_bids_in_auction_median: 4766
time_delta_bids_in_auction_kurtosis: 5714
total_bids_in_auction: 1175
first_auction_bid: 2
last_auction_bid: 2


In [10]:
df_from_csv.head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,outcome,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid
0,0,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.157894736,0.000000,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,1,0
1,1,25,668d393e858e8126275433046bbd35c6tywop,38v0a,furniture,phone4,9759243315789473,bd,147.64.206.30,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.315789473,0.000000,3.931033,4.550885,0.0,0.947368,2.421053,5.210526,45.052632,2.421053,11.106156,3471,1,0
2,2,39,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone35,9759243368421052,in,75.11.239.20,i92xwb95qdo901j,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.210526,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,0,0
3,3,42,668d393e858e8126275433046bbd35c6tywop,jkyzt,furniture,phone38,9759243368421052,in,215.203.78.242,s4hrzaqd5vbq2he,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.000000,3.523134,4.137105,0.0,0.842105,2.105263,4.631579,60.473684,2.105263,17.341241,3872,1,0
4,4,68,668d393e858e8126275433046bbd35c6tywop,h2xrf,furniture,phone57,9759243526315789,in,134.198.237.242,1euklxshlldto4z,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.526315789,0.000000,2.897465,3.318102,0.0,0.684211,1.842105,3.894737,31.000000,1.842105,10.754362,4709,1,0


## get single bidder to test with

In [12]:
df = df_from_csv.loc[df_from_csv['bidder_id'] == '668d393e858e8126275433046bbd35c6tywop'].copy(deep=True)

# add temporal difference between bids
df['date_time'] = pd.to_datetime(df['time'])

# must sort by date time and index are not in same order
df = df.sort_values(['date_time']) 

tdelta = df['date_time'].diff()
tdelta.fillna(0 , inplace=True)
df['time_delta_user_bids'] = pd.to_timedelta(tdelta) / np.timedelta64(1, 's')

df.shape

(19988, 28)

In [13]:
df.head()

,index,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,payment_account,address,outcome,date_time,time_delta_auction_bids,time_delta_auction_bids_mean,time_delta_auction_bids_std,time_delta_auction_bids_min,time_delta_auction_bids_25%,time_delta_auction_bids_50%,time_delta_auction_bids_75%,time_delta_auction_bids_max,time_delta_bids_in_auction_median,time_delta_bids_in_auction_kurtosis,total_bids_in_auction,first_auction_bid,last_auction_bid,time_delta_user_bids
0,0,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.157894736,0.000000,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,1,0,0.000000
1,1,25,668d393e858e8126275433046bbd35c6tywop,38v0a,furniture,phone4,9759243315789473,bd,147.64.206.30,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.315789473,0.000000,3.931033,4.550885,0.0,0.947368,2.421053,5.210526,45.052632,2.421053,11.106156,3471,1,0,0.157895
2,2,39,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone35,9759243368421052,in,75.11.239.20,i92xwb95qdo901j,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.210526,3.455380,4.189127,0.0,0.842105,2.210526,4.526316,70.315789,2.210526,28.351440,3949,0,0,0.052632
3,3,42,668d393e858e8126275433046bbd35c6tywop,jkyzt,furniture,phone38,9759243368421052,in,215.203.78.242,s4hrzaqd5vbq2he,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.368421052,0.000000,3.523134,4.137105,0.0,0.842105,2.105263,4.631579,60.473684,2.105263,17.341241,3872,1,0,0.000000
4,4,68,668d393e858e8126275433046bbd35c6tywop,h2xrf,furniture,phone57,9759243526315789,in,134.198.237.242,1euklxshlldto4z,a3d2de7675556553a5f08e4c88d2c228ucoac,42a3b61a1fe69d66ad60f3e347aa09b1erfe2,0.0,1970-04-23 22:54:03.526315789,0.000000,2.897465,3.318102,0.0,0.684211,1.842105,3.894737,31.000000,1.842105,10.754362,4709,1,0,0.157895


In [14]:
df[1:]['time_delta_user_bids'].median(), df[1:]['time_delta_user_bids'].mean(), df[1:]['time_delta_user_bids'].kurtosis()

(0.421052631, 0.6825357535029493, 16.296545126730702)

In [15]:
# get describe series (not including first row delta will always be 0)
describe_of_time_between_bids = df[1:].describe()['time_delta_user_bids']
describe_of_time_between_bids

count    19987.000000
mean         0.682536
std          0.833721
min          0.000000
25%          0.157895
50%          0.421053
75%          0.894737
max         10.315789
Name: time_delta_user_bids, dtype: float64

In [16]:
# get axes labels for label describe
describe_axes = describe_of_time_between_bids.axes[0]
describe_axes

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')

In [17]:
# add distrubtion of deltas data
for i in range(1, len(describe_axes)):
    df['{}_{}'.format('time_delta_user_bids', describe_axes[i])] = describe_of_time_between_bids[i]

In [18]:
bot_ip = set(bots['ip'].unique())

In [19]:
len(df['ip'].unique()), len(bot_ip)

(10316, 230125)

In [20]:
count = 0
for ip in df['ip']:
    if ip in bot_ip:
        count += 1
count / df.shape[0]

0.48789273564138486

## slice df by bidder_id

In [21]:
bidder_dictinoary = {}
bidders = df_from_csv['bidder_id'].unique()
count = 1

for bidder in bidders:
    bidder_dictinoary[bidder] = df_from_csv.loc[df_from_csv['bidder_id'] == bidder].copy(deep=True)
    
    print('count: {}, id: {}, df_shape: {}'.format(count, bidder, bidder_dictinoary[bidder].shape))
    count += 1

count: 1, id: 668d393e858e8126275433046bbd35c6tywop, df_shape: (19988, 27)
count: 2, id: e8291466de91b0eb4e1515143c7f74dexy2yr, df_shape: (30062, 27)
count: 3, id: a58ace8b671a7531c88814bc86b2a34cf0crb, df_shape: (192565, 27)
count: 4, id: 7e34a26c4c339e875997f4b9239d2b5fscng4, df_shape: (9108, 27)
count: 5, id: 6c8ffec692e88d81e1b18d47818abb04v6llr, df_shape: (84319, 27)
count: 6, id: 37bf6f23b628a3e2b5b22ba81beccbef0efoh, df_shape: (6355, 27)
count: 7, id: af9c96944265cf541b3fe734a057821a825l7, df_shape: (2390, 27)
count: 8, id: 52d8066e28f168e5bee769559ead307e1go7e, df_shape: (5189, 27)
count: 9, id: 50a4bfbac8a9653af4099ec1169f684c2nsgu, df_shape: (4429, 27)
count: 10, id: 76f17ce227e593aeff456fed46e36c83a3j72, df_shape: (3379, 27)
count: 11, id: 46a28eaed47c29b4fd148a50cde44840u0bsn, df_shape: (29146, 27)
count: 12, id: cffb5f3c1f4ca815d0f643fcb2f2f316beod4, df_shape: (51421, 27)
count: 13, id: dd055846eb553ab8e953b084f2048d53bdqec, df_shape: (48018, 27)
count: 14, id: b1863365ae2

count: 111, id: b7e16fd4fd9ff04b7d741b70730b7b6b1aw84, df_shape: (743, 27)
count: 112, id: 48badabcce6015d257ff5809761486c8qgjul, df_shape: (2336, 27)
count: 113, id: 90fe8d6fc0bbe4ff2ca7833fe51bddbaaibof, df_shape: (1912, 27)
count: 114, id: 01067975436d123f717ee5aba0dd4bbfa0937, df_shape: (543, 27)
count: 115, id: b0164d9a18a3cdf6082e96f9708040690qcqf, df_shape: (255, 27)
count: 116, id: 3e85a43950c71b1988a814682abd4392yakvp, df_shape: (819, 27)
count: 117, id: 0a43de0ab0a7525fec59ef4bc1dda7a2z9z69, df_shape: (743, 27)
count: 118, id: 18bc9dbbed75b55f35ec4be1351b7aa86an68, df_shape: (985, 27)
count: 119, id: 92f8392be617527734a2f18482815af9ak8l3, df_shape: (1503, 27)
count: 120, id: ea57e35acda9a8f8b75f4d9bc049f74d7qx3t, df_shape: (137, 27)
count: 121, id: 069d251f2fc19102924468ab38349d5eou7ab, df_shape: (13, 27)
count: 122, id: 08515611e73803dc6d8c562b8514dfb5qi3z9, df_shape: (297, 27)
count: 123, id: 669c6a0e8aabbf0647e8ee6e150c47b18a4pu, df_shape: (252, 27)
count: 124, id: c749a25

count: 221, id: 0e66933c924764a59760aaae5368e7d1e0s88, df_shape: (57, 27)
count: 222, id: 2918b09c1ed9e9e8602f85175b266aeej5v6w, df_shape: (86, 27)
count: 223, id: e67abc3ba39537cc640678297a7c5bfag490k, df_shape: (143, 27)
count: 224, id: 99bcfc00be736e03facf0afeb79ac533cs7tp, df_shape: (257, 27)
count: 225, id: dcd54d7afa6820e3f00e1c9e83aecc7b4f3lw, df_shape: (217, 27)
count: 226, id: 8b253bd318229b3eea3f459bd81b8347khytb, df_shape: (494, 27)
count: 227, id: 054b51d059fe981ca8cc4e0d4c49aabb5mck7, df_shape: (1474, 27)
count: 228, id: 6086425310e05e77a53317490e8c1f3cxzf2r, df_shape: (373, 27)
count: 229, id: 26508777dcd84df59b467a60d9f18ca1pvvbs, df_shape: (159, 27)
count: 230, id: b4495464d4503ccb8e696994059983c89g4lc, df_shape: (36, 27)
count: 231, id: 9a12aa1aa3560062c6f84bd374eb5f7f422vq, df_shape: (26, 27)
count: 232, id: cba6f45915ebbd4b5404c6d357153c0adeblt, df_shape: (249, 27)
count: 233, id: c2d1b8c1b7a9c8010f5b215e8aadb1d7gahpp, df_shape: (226, 27)
count: 234, id: cbc71ae3c369

count: 332, id: 916aedb8fd708e100318969dfe57a4639hexd, df_shape: (47, 27)
count: 333, id: ba194af5111cfcf1dd95eb36160c728ds4kcy, df_shape: (13, 27)
count: 334, id: 9b51afa3c3a736bdf93bd1f2a7b0a0903c34w, df_shape: (133, 27)
count: 335, id: 67a1b129ca241c51d0abb5484f8eb3fct8o5c, df_shape: (10556, 27)
count: 336, id: f9753e9fd6e25e0a1ae176ab95c31146iaqnv, df_shape: (25, 27)
count: 337, id: e0a9e4ffe2acd28599dacb91c98c34bc66vo3, df_shape: (68, 27)
count: 338, id: 4d1542974bc322859d3d099b8c5193felot1k, df_shape: (335, 27)
count: 339, id: 354d4abc23881a40b8f119225e0dbd09tk1u3, df_shape: (32, 27)
count: 340, id: 462df7ecf1d47980dea6a93d44fb408d4u2d5, df_shape: (82, 27)
count: 341, id: 37ca0d0ded383d709676a4f9729ef39cpjfh4, df_shape: (14, 27)
count: 342, id: 782557b3ec17063f2a36fc9193ed3200h14me, df_shape: (257, 27)
count: 343, id: 050a3b29ca1818e1f237df8132bc2013bcuxi, df_shape: (40, 27)
count: 344, id: 8a958e0092f6ca984107ea6c3975f806c59m2, df_shape: (3, 27)
count: 345, id: 438e53670ffa2990c

count: 443, id: 24a9abf72cedcb421e831c1be26607e37zojy, df_shape: (14, 27)
count: 444, id: 1b7c070ffb2c23c1d02ae5bc5d817d06qz4xb, df_shape: (8, 27)
count: 445, id: 0817731a4d2e319f6f771da3f522b8bfv6njs, df_shape: (224, 27)
count: 446, id: 0e8212e286ee36da8945c2480e366417xa22x, df_shape: (3, 27)
count: 447, id: 0e373556d3bf588a1bb2ca6354120cf2dmvjs, df_shape: (3, 27)
count: 448, id: f219cbb3d454b32953619d5a4551f56dcg7gy, df_shape: (8, 27)
count: 449, id: dfa7ead5ef84eebc79c67d3daac340b0exa41, df_shape: (3, 27)
count: 450, id: 6c3bdc508c20f2f3871766cd73064c8cib3xh, df_shape: (48, 27)
count: 451, id: 14bcf558addd6e6f2818ab2bda7dc679jqoon, df_shape: (36, 27)
count: 452, id: 18b5e850a1c10e681cb5fae0704a699018x6f, df_shape: (1, 27)
count: 453, id: 36473d4c641c4c09c20c80e2ab35a36dwzp5d, df_shape: (6, 27)
count: 454, id: a3d0a600cdb44f03d58f27b19247cd0d4az4k, df_shape: (3, 27)
count: 455, id: 3d94b376bae8dd60eea48ded36134a3962glw, df_shape: (9, 27)
count: 456, id: 9685887173cc2e1e5ee3993832a4b1

count: 666, id: e8e4603ed284adbd252a7a601f9bffe09oq6f, df_shape: (8, 27)
count: 667, id: eefb0b863345336dce70312ff0f8faeb9qksv, df_shape: (2, 27)
count: 668, id: 65435e491aa69dd9d44d70a9aeb6e847oqlty, df_shape: (5, 27)
count: 669, id: 69e97e52dd6428671f46f2ab6567172d8mfar, df_shape: (1, 27)
count: 670, id: 36e2e2f32a3872d3e958c0a58ee79bd129xdy, df_shape: (10, 27)
count: 671, id: 6ecd9f4b332eb790dc933aea0a1528e2qkp4y, df_shape: (4, 27)
count: 672, id: c806dbb2decba0ed3c4ff5e2e60a74c2wjvbl, df_shape: (2, 27)
count: 673, id: 61101089c1b34f409fc9b836a7f9cd163y22l, df_shape: (2, 27)
count: 674, id: 2b4b36fd77eaeabbdad8fac68b17ece5ey04u, df_shape: (1, 27)
count: 675, id: 3e2e03a4464642627e223fba066182d1qslv5, df_shape: (2, 27)
count: 676, id: 8f00a4c4d7f097bc1688f8d89431f6bfmodq2, df_shape: (1, 27)
count: 677, id: e8bc0150097c1b2b8b8c374b69e0d3a6mxfcm, df_shape: (2, 27)
count: 678, id: 222a0e05442fec0af2d20ebdcafe4b28c2pi4, df_shape: (4, 27)
count: 679, id: eeda0d135495167db9425fc5fa3a5e3ex2

count: 778, id: 8cba56e5e7c2be942ea8472b73f0842fil1x0, df_shape: (7, 27)
count: 779, id: 4694d67c21e9a2edd614e717aa3f4dd2e0w93, df_shape: (1, 27)
count: 780, id: 44c38c27b9de757c8c7c05cf07b7e3eb14qpt, df_shape: (1, 27)
count: 781, id: 1880a778c9dd0cb15ca1f052308a8e04g7e35, df_shape: (3, 27)
count: 782, id: 2db8669a6d21a5c8197d4287ea32b7883gv5r, df_shape: (5, 27)
count: 783, id: 1e1c9332e90c1cb0f01da223bc043c528cvwo, df_shape: (2, 27)
count: 784, id: 3ef757c3d6a272681c69bbd7e071b705yzt8t, df_shape: (3, 27)
count: 785, id: 817f8074bad1c717afaa8d1696f9b2b00muzw, df_shape: (2, 27)
count: 786, id: 65659f3fbf6207c570c4bb8ac64772e9umh95, df_shape: (1, 27)
count: 787, id: ae15accae2164aae084aef7e3524c1fflq962, df_shape: (3, 27)
count: 788, id: 332ef2c7189603a497fc89345ee4f5605i5xw, df_shape: (4, 27)
count: 789, id: 900284c412387e0bb2dc2aa5ce42dbb9th70g, df_shape: (2, 27)
count: 790, id: fba0c76970a464cfa9c043c906442f0eg53a2, df_shape: (1, 27)
count: 791, id: 28b4c35bd90c81ca959da328806469d493m

count: 996, id: 5720c8d7bc15b94037cd75ff9496a35evugeo, df_shape: (1172, 27)
count: 997, id: ec0c9e98cdf302274974ad212477f851iw35b, df_shape: (2984, 27)
count: 998, id: d6fd864297177c38cea65aed8a45fb7bbp2e3, df_shape: (1645, 27)
count: 999, id: b729f60d26fde89a01f3fa89759d6b07t0ec4, df_shape: (2500, 27)
count: 1000, id: 8cb96eaadc11cf4f914bad84518632e7czdyl, df_shape: (2500, 27)
count: 1001, id: fa64831ab5ade55daee13f852c16b0a84ujt0, df_shape: (888, 27)
count: 1002, id: d42604c7cca8ec92839e92508f2a3e52ehz29, df_shape: (4624, 27)
count: 1003, id: b4550ab1da3a3ed6e4a206c9ef78d496q23hb, df_shape: (889, 27)
count: 1004, id: 651402deb28af3e3d1ab9a6b88314b591uuaz, df_shape: (7, 27)
count: 1005, id: ba266f4898bf572447ad65856b362595yjt2a, df_shape: (652, 27)
count: 1006, id: 679d60378d42ca9d3b8a28073fe14fc7bv4jh, df_shape: (455, 27)
count: 1007, id: 6e29de28296a57f039026ffc6df9145cxo2n0, df_shape: (878, 27)
count: 1008, id: a7bc17dfb615eacffe397be5b2c2f3a6r0bb9, df_shape: (4056, 27)
count: 1009

KeyboardInterrupt: 

In [ ]:
len(bidder_dictinoary)

In [ ]:
bidder_dictinoary['668d393e858e8126275433046bbd35c6tywop'].head()

## add features generated from testing on single bidder to all bidder

In [31]:
path = 'user_bids_dfs_training'
# path = 'user_bids_dfs_testing'
i = 0

bot_ip = set(bots['ip'].unique())


for bidder in bidders:
    
    df = bidder_dictinoary[bidder]
    
    df['date_time'] = pd.to_datetime(df['time'])

    # must sort by date time and index are not in same order
    df = df.sort_values(['date_time'])

    # add temporal difference between bids
    tdelta = df['date_time'].diff()
    tdelta.fillna(0 , inplace=True)
    df['time_delta_user_bids'] = pd.to_timedelta(tdelta) / np.timedelta64(1, 's')

    # get describe series (not including first row delta will always be 0)
    describe_of_time_between_bids = df[1:]['time_delta_user_bids'].describe()

    # get axes labels for label describe
    describe_axes = describe_of_time_between_bids.axes[0]

    # add distrubtion of deltas data
    for i in range(1, len(describe_axes)):
        df['{}_{}'.format('time_delta_user_bids', describe_axes[i])] = describe_of_time_between_bids[i]
        
    df['time_delta_user_bids_median'] = df[1:]['time_delta_user_bids'].median()
    df['time_delta_user_bids_kurtosis'] = df[1:]['time_delta_user_bids'].kurtosis()
                                              
    count = 0
    for ip in df['ip']:
        if ip in bot_ip:
            count += 1
            
    df['bids_with_bot_ip']  = count # bids that shared ip with bot
    
    df['total_bids'] = df.shape[0] # total bid count
                                              
    user_ip = set(df['ip'].unique())
                                              
    df['ip_shared_with_bots'] = len(bot_ip & user_ip) # number of unqiue ips that are bot ip
                                              
    df['unique_ip'] = len(user_ip) # unqiue user ip
                                              
    df['avg_bids_per_url'] = df.shape[0] / len(df['url'].unique())
    
    df.to_csv('{}/{}.csv'.format(path,bidder), index=None)
                                      
    print('biders saved: {}'.format(bidder))

/home/luis/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


biders saved: 8dac2b259fd1c6d1120e519fb1ac14fbqvax8
biders saved: aa5f360084278b35d746fa6af3a7a1a5ra3xe
biders saved: 3939ac3ef7d472a59a9c5f893dd3e39fh9ofi
biders saved: 8393c48eaf4b8fa96886edc7cf27b372dsibi
biders saved: eef4c687daf977f64fc1d08675c44444raj3s
biders saved: ab056855c9ca9d36390feae1fa485883issyg
biders saved: d600dc03b11e7d782e1e4dae091b084a1h5ch
biders saved: fe37b8b29d26021ea9462bcc7c5bf5c2pbya7
biders saved: 990db0dcdb665fb8fda5845b3e5d63082mbqq
biders saved: 1a9c554b044c77239b63636008d353d4vy9kh
biders saved: ffbc0fdfbf19a8a9116b68714138f2902cc13
biders saved: c4856fd5abe8f6d6dea36ca2fec444faauos8
biders saved: d174177146370aacbd5ae107b6be78828tx3e
biders saved: 6b09f3376045ebad1504abe3b05c9c943aezq
biders saved: d47110cdd3eeb1ec25659f4285730e0a28ofd
biders saved: c709b6c05ebd88124fe639a70103c8aerho8s
biders saved: 0dea01e325acccb9d2e7cdbf0150619c6aycy
biders saved: 708ba5d36271639a9003ace4dc2cc540mugek
biders saved: 45972f59df79953d9bcf04a76dbe0f51z23qd
biders saved

biders saved: 6c4f92086930144fc55a79dc0a4a6f339zt0p
biders saved: 47efdea29f084d75516f475ee2e21695ira9g
biders saved: 48fd18427f5e4b8d9cb568aa5cd6a3f9sizo0
biders saved: 0cea6e30d0b3e33b2a77d68b8c8e1ab3vzomo
biders saved: a51eb0674d176d1601c239e290d8db9aygqcy
biders saved: bfba91e586b1c37429d1e40ffb5febfagbbxn
biders saved: 5e9beac339e73e3feb340f9372b4aa07l69vw
biders saved: 0565d7d345ac29442b26cd9ffe2b3b841wwnb
biders saved: b569644e65d437c29a86d6c86f86c720gieux
biders saved: 84c3f1ef08df383bdc98f5465e885b6edepz9
biders saved: 55dc05893025427ce0aa0ffce381a2bb5tlqu
biders saved: e0fa75eb43b7bd390221db66926f7563hgapj
biders saved: c02f06b962414bde7331252da5e0ee7dbhxwl
biders saved: cb08091907558694c82474f05fa81567pdtie
biders saved: b57f6875839b5a6ce409198cab6d3d08zoydm
biders saved: 59f61cb410ac321a795c4d5a388e787aqsytg
biders saved: 1a1d4e9eba20c58b73fd68c5ab9907081xw9l
biders saved: 31c469698f66a5d79f5984ab04429837z5ype
biders saved: a64afbb3c053e8cccc58ce09a2e71a5205u6z
biders saved

biders saved: c671c7faec19e9409eed917213831d0015juj
biders saved: b766fbc1d5d0525e18b094162b543b15ub8qf
biders saved: 24d0694fbff381cdd9f40301080301d8yngb7
biders saved: 51f47f594971d59e2a52398527ce8d81q5rdo
biders saved: 6ceea2e62f9c4f03f1a4efa7f8d2b2416hers
biders saved: ecf6ad2bbfa8aafbd7ad51d5bfcc02ddyznwh
biders saved: 78274148071dfc23b583be546d1e80c7lu8rq
biders saved: c1e8091db6fd2c7977b15830e0efc4012zjt0
biders saved: b4259cbae2d1542752b49b96c544be2bxp8u2
biders saved: 02518a364d87661eed1453e251933862yvmjj
biders saved: 0764308c5a1d35bf76c32d943eff07a50tp9e
biders saved: ab7cc752f0ed98d7f8c85e02dbbc4772z9xpx
biders saved: 7e4a99f7ae5fe3a098c9c3f21b9a5cbfa4w4t
biders saved: 3de7d3030fecf543874081f7ab0a248evqper
biders saved: b8fc67ed5825409a40adca46ee7f053auho77
biders saved: 6951db6168b80060a103ab65d0e074484c1vc
biders saved: 9a51d309c8824bae38bd75eddce1a99fe208r
biders saved: b02df28d2798f0b6c2b1112a9d407ae7wbj8r
biders saved: a9c0cd61e6b52a5b4bfb88e5ccb40af36qzu2
biders saved

biders saved: 64b2a3d72fd485989bff7ee11bf66f7b51j0z
biders saved: d8b3e32a6f8436acfe9984227c9d1e57qemx2
biders saved: a351fbee43a761cef22150a06645b52c1k104
biders saved: 336f98d51f267347d9a751d1d7b97355nw7xc
biders saved: 58ea3d35765cb8ceeb6b4be54d754ea0bpguj
biders saved: fe84da5e6912534549d63415ed725618uiz26
biders saved: 554a2b2d9728c432126398b45fe804e6ega9f
biders saved: f8e3d1df88d8cf283a58d0a2329b79a9ilvmp
biders saved: a97593cabdfc92d0ea142d1c64f0cb70aa6cs
biders saved: 33dc4889cc2bd50dab02344d6fe19a1aj59pt
biders saved: 51d6b173586fd34c938477d87ad6b6b0092ke
biders saved: 51b2d173034d54f115b647ac7c1640a5l1q91
biders saved: 3cccf376d4318df23f4bd9321cd83cf0ys9lz
biders saved: 6264a918aa934905a79a167af1d0787e7om3b
biders saved: 8a2b950ec6f3c73253cd80133d184cdawzip0
biders saved: 2974f614bc988411be383b2d6d52003fdb1ps
biders saved: 7fe504252783ec1fd2b203d3de7040e6lflsb
biders saved: f8e215d416736fdb1de26d34ad315c368o24w
biders saved: 59998f2c7299efed20dbb68df23a53c1cdoje
biders saved

...

biders saved: c368adc1be8ffd5a95d526b24b6c2fdbg3wmi
biders saved: 6d1cf391edb3f05c26770c4c0b8062680u4gh
biders saved: b3dd575821661bec07367a083af2bceb70717
biders saved: 877eecc4f4097dad7de7fa6c4fbd0ab5ncsl6
biders saved: a91987c152eacd9de8a58efeb950e5a1leuxk
biders saved: ae96be6b28d9cf2fe960d8e14a8ca73fjilb0
biders saved: 1240be25625da5ff66a410ed715d2e8co6esd
biders saved: 23fa8e1bbc080f712b2a135fe0d8bfb426w7l
biders saved: 18c36ff4ba3fd016bbaf516b05e4bb5a1og5x
biders saved: e073cd48ba0f6f9b7d6a983cb260f8f3miffj
biders saved: 7ee44eefd8cb3712e51c8a91c3fd0c21ckpgj
biders saved: 3fa481a797c93264686ceae13400848bv5izy
biders saved: 11d0e15d71306a034cbf406d8cd38908sxgom
biders saved: 802ad343f940c4eaa7e3ba6f2b755fcccybto
biders saved: a111bba2185668cbb1c753aa4e252e7307pcq
biders saved: ebf866e713e4a606301f2ccd3d996fdav38b5
biders saved: e2f091617f5dadd027282936254a9d58re0sn
biders saved: bf3dd184ce9855178cdb50155030d15atdhb5
biders saved: 5b1893e2a75a2ffd4272d75a6aab6047yee7o
biders saved

biders saved: 91cb97491f3deb60457041ad665a642720z5z
biders saved: 8506033bad90a03d59f849054e5a0b91a5gel
biders saved: 601fffaf4ead097c9dc9c40a2eecb505lcs19
biders saved: 620492932ce60827664b417eed6c83d6b1ky0
biders saved: 38c2be4dfa8f0512b3164fbbf6ffb1f6weo0v
biders saved: 82be315206c48aacefccbb7e7da192e2q18am
biders saved: 038d5c12a210df3818eb8dc34ebedac2ldr62
biders saved: 5a12f5a72db38165a610f73387c29ad9v0atb
biders saved: 928996e2dbbae8694d3650587cb49b1ahqbaz
biders saved: fb294f3671fc19eacf5f13d3a6c5a795pzlxm
biders saved: 8c07083331a97da1b8a99cea75775401m0a0r
biders saved: 75351c0b43c8732ed763e9ebc7a3241do08lc
biders saved: 4f8188d297e19e277a2dbafbecdeb53crpf4h
biders saved: 42d68de5fac9c929d58d277a6f1287e0qbql4
biders saved: ade3651ac046baabe9a0d2d4fc587320tlsd1
biders saved: ad69c8ef95b0ca2ad4ab8f9135001030xswpf
biders saved: 1f52c4c6fffd6b5d0f89126c2fa0b45a1l33z
biders saved: f4832b1f7ea28da0093ba42b06e38aa2p386e
biders saved: a8217f09ac37688d06a46dde0a443564dn51e
biders saved

biders saved: 4ccde027a20264ab2d04392b60e31b4fu3ray
biders saved: a356163fe7434d47b1c6480c87366394oaibe
biders saved: da8abe1879eb50004c6276c94b792e11lq4xn
biders saved: a852b00d87721f8d3ca2e1041c088aa507iso
biders saved: 58cbeb46c36b90a723cded01495d055bmwudo
biders saved: 10a1f1cb1fdd93a139efba25087dbeeaulogd
biders saved: 980a1950c0b5df2b9ed03cb05e657848p76xr
biders saved: 992737f67ac0150c8e1448bdc06e3d2f6f57s
biders saved: d88f8b2015441d1ca588f5bc850629040hdtv
biders saved: 54f0943d5e936acdef05bf151ffbb1eczg2ck
biders saved: 8950b00903c06ced5253da31b99d80fe9suie
biders saved: 5ba001c8d950b2b21c465f1fc5e40630qy65o
biders saved: c28049769ec09eb046314cffba0975cfqvkyv
biders saved: b7e9ad9b90c4b148a9ddd7815497f8395rib9
biders saved: cc1b98265a249b0373d3014fc504b100cl1ja
biders saved: 628571f862245c6a83623648b5543d087nfqx
biders saved: 1eb1f5561dd35f544202c3b509d578422058h
biders saved: 740cce529e3b68bb776f3ae276b959bary8ax
biders saved: 0933e1844ef295b026b5e6ff6f4c0960jhzqr
biders saved

biders saved: fdd71714ea1426c36bbc80a157f50c8daex1p
biders saved: e8a1dc9fb527faf420bd7e5196d875cfed9id
biders saved: e154f640ac6b7e56dc4eaa610a283d8e3l2km
biders saved: 601bbfc2d687f56d1b18c99ffde76c7a94ek8
biders saved: 1170c87a5f74dccb65eb064aaffe3a29ioc7n
biders saved: bb75daa2552fa7e03972a7ebd49c8270dxs87
biders saved: 242e4323f41d0a9011b4140ba74f478dnfsqs
biders saved: 1960e29a911e5a2dbc7eba2b373c1e17tjpd0
biders saved: 288452bc92ad126755ead4406718da67e1d38
biders saved: 774e41491bfb8338b75016cf9fba5d9e6oho5
biders saved: b972f198aad37412a797c0e9b5ef232ac2fhl
biders saved: 222a84e78164c6fbe3cc7e7fc25c16b2jqncf
biders saved: 06052ab145f8a3ddc6ad8433ef11da8dputmu
biders saved: bd612f236a50b7028858ec6a9518dcf82wdtq
biders saved: c979898ec4012f0933c38a6bec19c92cvmlq6
biders saved: 05f5792216f9a5bfe91ecd0dbccc3cdceuwmf
biders saved: 016e3c511caf01a845c9c976bbf355a6m1nns
biders saved: 6a7fc08059bc5d4ea71f7a00db4d6756xuba2
biders saved: 5ab20c9699e9ea7c9b395967027b3245f4uq1
biders saved

biders saved: 6887f0abc4eb4c79eb0e23c48ceea186vjfih
biders saved: 5455e2e995dbd8e57ced16096e5a2769lmthu
biders saved: 605520624bbf2020903a32aecd36e535jgj0h
biders saved: 2d0703227815eb74617f639b13460351i3s9l
biders saved: bec86221a3ab9f5b8f502a4bed42f2cfddf3p
biders saved: 1b2e5adebf42f673c117d008cc713ff1x6cs4
biders saved: 0751502caf4afd959c99955c80edacc85u11v
biders saved: 61665d0c0ac353aa4f429f636879af8bmuf9n
biders saved: 3a2e6d8da93da7b6f5b9c6ba770f8687tfkyk
biders saved: 1af3a143e126acfc43031dd8e4612b2egr927
biders saved: 83b6cbf8c26ea604b48f508dd7728c9414pfn
biders saved: bd890937b5601111119969b45bde5056eycor
biders saved: 107f0f8f2bf4fc62ba8c01003c516c03npt0l
biders saved: 9f9313a1d8a88b3a3c8a2d893a92820dn38qc
biders saved: 5563598101c7e6533b63f26827e57aa96qjww
biders saved: 15e7068e841df3f00857b9b1db46e79200trv
biders saved: 5a8019da4075c77405e43efd072670eegbme0
biders saved: 9390ac0a97e75b3b56fe63695d27a82euhv69
biders saved: 13eaa629e1a8fd178403a66fcf0758bbirawk
biders saved